In [ ]:
!pip install datasets
!pip install tokenizers==0.10.0rc1

# 🤗 Tokenizers

Для работы с текстом нам нужно уметь представлять его в понятном копмьютеру виде и первые шаги в направлении - это разбить текст на токены (которые могут быть словами, а могут и не быть в зависимости от метода токенизации) и сопоставить каждому слову уникальный номер от 0 до размера вашего словаря. После этого мы сможем конвертировать строку типа "это токенизированная строка" в вектор вида `[47, 392, 38]`. Эта операция - замены токенов на их индексы - часто называется **нумерикализация**, а обратная операция - **денумерикализация**.

Если вы используете модуль `sklearn.feature_extraction.text`, например для векторизации с помощью `TfidfVectorizer`, sklearn выполняет все эти операции за вас под капотом, но если вы хотите использовать нейросети, жизнь становится гораздо сложнее и до недавнего времени почти все использовали самописные штуки, чтобы (де)нумерикализовать тексты. Это создавало множество неудобст связанных с тем, что во-первых вам приходилось писать больше кода (и создавать больше ошибок), во-вторых рядом с нумерикализацией лежит очень много мелких подзадач включая использование специальных токенов, таких как UNK, сериализация ваших объектов, и распараллеливание на несколько потоков для ускорения. 

Huggingface Tokenizers постепенно становятся всё более и более популярными в NLP-сообществе. Эта библиотека быстро развивается и позволяет вам использовать различные методы токенизации - от разделения на слова по пробелу до разделения на битовые представления unicode-строк в зависимости от частоты битовой n-gram. Также она умеет делать нумерикализацию и очень быстро работает. Вместо самописных подходов, которые всё ещё часто преподаются в курсах по NLP, мы будем использовать 🤗 Tokenizers в течение всего курса и довольно быстро вы научитесь пользоваться даже не самыми очевидными её методами. А сейчас мы начнём с самых основ.

In [ ]:
import datasets
import tokenizers
tokenizers.__version__  # should be above or equal 0.10.0rc1

'0.10.0rc1'

За функции предобработки текста, разделения его на токены и нумерикализации в 🤗 Tokenizers отвечает объект `Tokenizer`. Самый наглядный способ его создать - это предоставить ему словарь, мапящий слова на их индексы. Давайте напишем функцию, которая это делает.

**NOTE:** Самая простая операция токенизации - разделение строки на слова с помощью regex - стоит несколько сбоку в Tokenizers и даже не называется методом токенизации, тк зачастую она используется как первый шаг более сложного метода токенизации. Поэтому она задаётся с помощью объекта вида `pre_tokenizer`.

Для этого мы делаем функцию, которая в начале создаёт объект `pre_tokenizer` и препоцессит наши тексты. После чего мы подсчитываем частоты наших токенов и выбирает top-N самых частотных, это удобно делать с помощью объекта `Counter` из встроенного в Python модуля collections. В игрушечном примере в слещующей ячейке, состоящим только лишь из двух коротких предложений эта операция неважна, но когда вы работаете с текстами размерами с Википедию, ваш словарь быстро станет необъёмных размеров и его нужно уметь огрничивать.

Третий шаг - создание мапинга слов на их индексы. Он довольно простой и его можно сделать в одну строчку с помощью dictionary comprehension. И после этого мы готовы создать объект `Tokenizer` и присвоить ему претокенизатор, который мы использовали для разделения текста на слова.

In [ ]:
from collections import Counter

import tokenizers
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace

texts = ['a list of sentences from my dataset', 'this is a text with known words']


def make_whitespace_tokenizer(texts, max_vocab_size=10_000, unk_token='UNK'):
    pre_tokenizer = Whitespace()
    tokenized_texts = [[w for w, _ in pre_tokenizer.pre_tokenize_str(t)] for t in texts]

    c = Counter()
    for text in tokenized_texts:
        c.update(text)

    token2id = {word: i + 1 for i, (word, count) in enumerate(c.most_common(max_vocab_size))}

    # usually, UNK is assigned index 0 or 1
    token2id[unk_token] = 0

    tokenizer = tokenizers.Tokenizer(WordLevel(token2id, unk_token))
    tokenizer.pre_tokenizer = pre_tokenizer
    return tokenizer


tokenizer = make_whitespace_tokenizer(texts)

# Encoding and decoding text

Для нумерикализации текста мы используем метод `.encode()`, возвращающий объект класса `Encoding`. Он содержит в себе много полезной информации, но самое главное из него - `.ids` дающий нам нумерикализованный текст, где слова заменены на их индексы.

In [ ]:
_text = 'this is a text with unknown_word'
e = tokenizer.encode(_text)
e

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
e.ids

[8, 9, 1, 10, 11, 0]

Денумерикализация делается методом `.decode()` (обратите внимание, что на вход к нему приходят `.ids`, а не объект `Encoding`). Эта операция очень важна на практике по многим причинам и одна из них - вы можете посмотреть на текст так, как его видит ваша нейросеть - с неизвестныи слоавми замененными на токен UNK и, возможно, с вамиши ошибками препроцессинга. Эти ошибки очень часты на практике и использование денумерикализации сильно помогает в дебаге.

Кроме этого нам не обойтись без этой операции в задачах генерации текста таких как машинный перевод, потому что на выходе из нашей модели мы ожидаем не последовательность чисел, а переведённый на другой язык текст.

In [ ]:
tokenizer.decode(e.ids)

'this is a text with UNK'

Вы можете также использовать аттрибут `.tokens` у объекта `Encoding` и он выдаст вам список токенов в этом предложении. Обратите внимание, что в нём хранятся оригинальные токены текста, следовательно его не стоит использовать для дебага.

In [ ]:
e.tokens

['this', 'is', 'a', 'text', 'with', 'unknown_word']

Ещё одна важная функция `Tokenizer` - это давать вам id одного слова или наоборот - выдавать слово по его id. Эти операции делаются методами `token_to_id` и `id_to_token`.

In [ ]:
tokenizer.token_to_id('this')

8

In [ ]:
tokenizer.id_to_token(8)

'this'

# Saving and loading the tokenizer

Если вы хотите задеплоить вашу обученную модель, вам нужны две вещи: ваша модель и пайплайн препроцессинга. `Tokenizer` по сути и является этим методом препроцессинга, тк он может включать в себя несколько шагов: нормализция и претокенизация, разбивание текста на токены и нумерикализация. Сохранение и загрузка `Tokenizer` реализована очень просто.

In [ ]:
tokenizer.save('tokenizer.json')

Давайте посмотрим как выглядит этот файл

In [ ]:
import json

with open('tokenizer.json') as f:
    tokenizer_json = json.load(f)

tokenizer_json

{'added_tokens': [],
 'decoder': None,
 'model': {'unk_token': 'UNK',
  'vocab': {'UNK': 0,
   'a': 1,
   'dataset': 7,
   'from': 5,
   'is': 9,
   'known': 12,
   'list': 2,
   'my': 6,
   'of': 3,
   'sentences': 4,
   'text': 10,
   'this': 8,
   'with': 11,
   'words': 13}},
 'normalizer': None,
 'padding': None,
 'post_processor': None,
 'pre_tokenizer': {'type': 'Whitespace'},
 'truncation': None,
 'version': '1.0'}

Мы видим, что токенизатор сериализоват как простой и читаемый json и содержит в себе претокенизатор `Whitespace`, словарь, указание на то, какой токен используется как UNK-токен и многое другое о чём мы с вами поговорим в будущих занятиях.

Для загрузки токенизатора используется метод `from_file`.

In [ ]:
loaded_tokenizer = tokenizers.Tokenizer.from_file('tokenizer.json')
e = loaded_tokenizer.encode('this is a text')
e

Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
e.ids

[8, 9, 1, 10]

# Batching

Ещё одна важная фича 🤗 Tokenizers - это возможность энкодить и декодить тексты в параллельных трэдах. Это становится очень важно, тк CPU-bottlenecks (когда предобработка занимает сопоставимое время с forward pass вашей нейросети) очень часты в NLP и могут сильно замедлить вашу тренировку, тк GPU будет ждать предобработки и ничего не делать в это время.

Для того, чтобы выполнить энкодинг в параллель, вы можете использовать метод `encode_batch`. Эта функциональность становится очень важной, когда размер вашего батча близок к 64 или больше.

In [ ]:
texts

['a list of sentences from my dataset', 'this is a text with known words']

In [ ]:
batch = tokenizer.encode_batch(texts)
batch

[Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [ ]:
batch[0].ids, batch[1].ids

([1, 2, 3, 4, 5, 6, 7], [8, 9, 1, 10, 11, 12, 13])

Если вы хотите больше познакомиться с 🤗 Tokenizers, мы рекомендуем вам почитать их [документацию](https://huggingface.co/docs/tokenizers/python/latest). 